# Data cleaning and exploration (EDA)

In [54]:
import torch
import pandas as pd
import numpy as np

import re

from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold


df_train = pd.read_csv('./data/train.csv', dtype={'id': np.int16, 'target': np.int8})
df_test = pd.read_csv('./data/test.csv', dtype={'id': np.int16})

print(f"Training set size {df_train.shape}")
print(f"Testing set size {df_test.shape}")


Training set size (7613, 5)
Testing set size (3263, 4)


## 1.1 missing value and duplicated rows

In [55]:
features = ['keyword', 'location', 'text']

print(f"Missing value ration in Training set:")
for feature in features:
    print(f"{feature} : {df_train[feature].isnull().sum() / df_train.shape[0] * 100 :.2f}%")
    df_train[feature] = df_train[feature].fillna(f"no_{feature}")


print(f"Missing value in Testing set:")
for feature in features:
    print(f"{feature} : {df_test[feature].isnull().sum() / df_test.shape[0]* 100 :.2f}%")
    df_test[feature] = df_test[feature].fillna(f"no_{feature}")
df_train

Missing value ration in Training set:
keyword : 0.80%
location : 33.27%
text : 0.00%
Missing value in Testing set:
keyword : 0.80%
location : 33.86%
text : 0.00%


,id,keyword,location,text,target
0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1
1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1
2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1
3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1
4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,no_keyword,no_location,Two giant cranes holding a bridge collapse int...,1
7609,10870,no_keyword,no_location,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,no_keyword,no_location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,no_keyword,no_location,Police investigating after an e-bike collided ...,1


In [56]:
# delete all rows with same text but different target, keep one duplicate of same text and targte
df_train = df_train.drop_duplicates(subset=['text', 'target'], keep='first')
contrast_target = df_train[df_train.duplicated(subset=['text'], keep=False)]
df_train = df_train.drop(contrast_target.index)
df_train = df_train.sort_index()
df_train.shape

(7485, 5)

## 1.2 Text cleaning

Urls are removed

Special characters (non ASCII)

Contractions are expanded

Punctuations #, @, !, ?, +, &, -, $, =, <, >, |, {, }, ^, ', (, ),[, ], *, \%, ..., ', ., :, ; 

Character entity references are replaced with their actual symbols


Finally, hashtags and usernames contain lots of information about the context but they are written without spaces in between words so they don't have embeddings. Informational usernames and hashtags should be expanded but there are too many of them. I expanded as many as I could, but it takes too much time to run clean function after adding those replace calls.

In [57]:
import re
contractions = { 
"aren't": "are not",
"aren't": "are not",    
"can't": "cannot",
"cant": "cannot",    
"can't've": "cannot have",
"could've": "could have",
"couldn't": "could not",
"couldnt": "could not",    
"couldn't've": "could not have",
"didn't": "did not",
"didnt": "did not",    
"doesn't": "does not",
"doesnt": "does not",    
"don't": "do not",
"dont": "do not",
"hadn't": "had not",
"hadnt": "had not",    
"hadn't've": "had not have",
"hasn't": "has not",
"hasnt": "has not",    
"haven't": "have not",
"havent": "have not",    
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"hes": "he is",    
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"isnt": "is not",    
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"lets": "let us",    
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"shes": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"thats": "that is",    
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"theres": "there is",    
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"theyll": "they will",    
"they'll've": "they will have",
"they're": "they are",
"theyre": "they are",    
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"wasnt": "was not",    
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"whats" : "what is",    
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"wheres": "where is",    
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"wont": "will not",    
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldnt": "would not",    
"wouldn't've": "would not have",
"y'all": "you all",
"yall": "you all",    
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"youd": "you would",    
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"youre": "you are",    
"you've": "you have"
}

In [72]:


def clean_text(text):
    # lower
    text = text.lower()
    
    # Urls and @username
    text = re.sub(r"https?://t.co/\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    
    # spaces, tabs
    text = re.sub(r"\s+", " ", text)
    
    # Contractions
    text = re.sub(r"\x89Ûª", "\'", text)
    for word, expanded in contractions.items():
        text = re.sub(word, expanded, text)
        
    # non ASCII characters
    text = re.sub(r"[^\x00-\x7F]+", "", text)
    
    # Character entity references
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&amp;", "&", text)
    
    # Words with punctuations
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        text = text.replace(p, f' {p} ')
    
    return text


In [75]:
df_train['text_cleaned'] = df_train['text'].apply(lambda s : clean_text(s))
df_train.to_csv('./data/train_cleaned.csv')
df_test['text_cleaned'] = df_test['text'].apply(lambda s : clean_text(s))
df_test.to_csv('./data/test_cleaned.csv')

In [76]:
from transformers import BertTokenizer

In [78]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sample = 'where is Himalayas in the world... map? @brother I\'m  not you\'re not aren\'t'
encoding = tokenizer.encode(sample)
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding))

[101, 2073, 2003, 26779, 1999, 1996, 2088, 1012, 1012, 1012, 4949, 1029, 1030, 2567, 1045, 1005, 1049, 2025, 2017, 1005, 2128, 2025, 4995, 1005, 1056, 102]
['[CLS]', 'where', 'is', 'himalayas', 'in', 'the', 'world', '.', '.', '.', 'map', '?', '@', 'brother', 'i', "'", 'm', 'not', 'you', "'", 're', 'not', 'aren', "'", 't', '[SEP]']


[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]